In [1]:
# %pip install -r requirements.txt
# %pip install crewAI

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

Setup Speaker Agents


In [3]:
names = {
    "AI visionary": ["tavily_search", "wikipedia"],
    "Grumpy old senior developer": ["arxiv", "tavily_search", "wikipedia"],
    "Junior Software developer": ["tavily_search", "wikipedia"],
}

Generate Discussion Topic


In [4]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

from helpers import llm

topic = "The current impact of automation and artificial intelligence on the employment situation of software developers"
word_limit = 50

topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}
        
        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the participants: {*names,}.
        Do not add anything else."""
    ),
]
specified_topic = llm()(topic_specifier_prompt).content

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Setup Tools


In [5]:
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.tavily_search.tool import TavilySearchResults

available_tools = {
    "tavily_search": TavilySearchResults,
    "wikipedia": WikipediaQueryRun,
    "arxiv": ArxivQueryRun,
}

Generate Agent Descriptions


In [6]:
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)


def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else."""
        ),
    ]
    agent_description = llm(openai_model="gpt-3.5-turbo")(
        agent_specifier_prompt
    ).content

    return agent_description


agent_descriptions = {name: generate_agent_description(name) for name in names}


def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
You can use the following tools: {', '.join(tools)}.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""


agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

Setup CrewAi Agents


In [7]:
from crewai import Agent


class DiscussionAgents:
    def __init__(self, names):
        self.names = names

    def speaker_agents(self):
        agents = {}
        for name, agent_tools in self.names.items():
            agents[name] = Agent(
                role=f"{name}",
                goal=agent_system_messages[name],
                backstory="You always respond directly to the actual discussion in your own way.",
                verbose=False,
                allow_delegation=False,
                memory=True,
                # tools=[available_tools[name] for name in agent_tools if name in available_tools],
                tools=[TavilySearchResults(max_results=1)],
            )
        return agents


discussion_agents = DiscussionAgents(names)
agents = discussion_agents.speaker_agents()
print(agents)

{'AI visionary': Agent(role=AI visionary, goal=Here is the topic of conversation: The current impact of automation and artificial intelligence on the employment situation of software developers
The participants are: AI visionary, Grumpy old senior developer, Junior Software developer
    
Your name is AI visionary.

Your description is as follows: AI visionary: A brilliant mind with an insatiable curiosity for pushing the boundaries of technology. You see a future where AI seamlessly integrates with human creativity, revolutionizing industries. Your passion for innovation is infectious, sparking excitement and debate wherever you go. Keep dreaming, and the world will follow.

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
You can use the following tools: tavily_search, wikipedia.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do n

Setup CrewAI Tasks


In [8]:
from crewai import Task


class DiscussionTasks:
    def speaker_task(self, agent):
        return Task(
            description=f"You are {agent.role}. You participate in a discussion. Always directly respond to the opinions of the other speakers.",
            agent=agent,
            expected_output="Output your opinion in 40 words or less. Do not output Sources.",
        )

Setup CrewAI Crew


In [9]:
from crewai import Crew, Process
from langchain_core.messages import ChatMessage


class DiscussionCrew:
    def __init__(self):
        agents = DiscussionAgents(names)
        self.speaker_agent_ai_visionary = agents.speaker_agents()["AI visionary"]
        self.speaker_agent_grumpy_old_senior_developer = agents.speaker_agents()[
            "Grumpy old senior developer"
        ]
        self.speaker_agent_junior_software_developer = agents.speaker_agents()[
            "Junior Software developer"
        ]

    def print_final_answer(_, intermediate_steps):
        if hasattr(intermediate_steps, "log"):
            log = intermediate_steps.log
            final_answer_index = log.find("Final Answer:")
            final_answer = log[final_answer_index + len("Final Answer:") :].strip()
            print(final_answer)
        else:
            return

    def kickoff(self, state):
        print("The discussion is about to start.")
        tasks = DiscussionTasks()
        crew = Crew(
            agents=[
                self.speaker_agent_ai_visionary,
                self.speaker_agent_grumpy_old_senior_developer,
                self.speaker_agent_junior_software_developer,
            ],
            tasks=[
                # tasks.host_task(self.host_agent, specified_topic),
                tasks.speaker_task(self.speaker_agent_ai_visionary),
                tasks.speaker_task(self.speaker_agent_grumpy_old_senior_developer),
                tasks.speaker_task(self.speaker_agent_junior_software_developer),
            ],
            verbose=1,
            full_output=True,
            process=Process.sequential,
            step_callback=self.print_final_answer,
        )
        result = crew.kickoff()
        output_messages = []
        for output in result["tasks_outputs"]:
            description = output.description
            role = description.replace("You are ", "", 1)
            role = description.split(".", 1)[0]
            output_messages.append(
                ChatMessage(content=output.exported_output, role=role)
            )

        return {"messages": output_messages}

Setup LangGraph Nodes


In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults


class Nodes:
    def __init__(self):
        self.tavily_search_tool = TavilySearchResults(max_results=3)

    def call_host(self, state):
        print("# Calling next speaker round")
        turns = state.get("turns") or 0
        turns += 1
        state["turns"] = turns

        return {**state}

    # Define the function that determines whether to continue or not
    def should_continue(self, state):
        turns = state["turns"]
        # TODO: change: if max rounds not reached, continue
        if turns <= 1:
            print("-- CONTINUE ---")
            return "continue"
        else:
            print("-- END ---")
            return "end"

Setup LangGraph State


In [11]:
import operator
from typing import Annotated, Sequence, TypedDict


class AgentState(TypedDict):
    topic: str
    messages: Annotated[Sequence[BaseMessage], operator.add]
    turns: int

Setup LangGraph Workflow


In [12]:
from langgraph.graph import StateGraph, END


class WorkFlow:
    def __init__(self):
        nodes = Nodes()
        workflow = StateGraph(AgentState)

        workflow.add_node("call_host", nodes.call_host)
        workflow.add_node("call_crew", DiscussionCrew().kickoff)

        workflow.set_entry_point("call_host")
        workflow.add_conditional_edges(
            "call_host", nodes.should_continue, {
                "continue": "call_crew", "end": END}
        )
        workflow.add_edge("call_crew", "call_host")
        self.app = workflow.compile()

Run Graph


In [13]:
app = WorkFlow().app
app.invoke({"messages": [ChatMessage(content=specified_topic, role="host")]})

# Calling next speaker round
-- CONTINUE ---
The discussion is about to start.
 [DEBUG]: == Working Agent: AI visionary
 

[{'url': 'https://www.brainerhub.com/blog/ai-in-software-development/', 'content': 'Brainerhub\nAI in Software Development: Role, Benefits, Impacts & Future\nAI has revolutionized the software development industry by enabling developers to automate routine tasks, streamline workflows, and improve the accuracy and efficiency of software development processes. It has made it possible to develop and deploy AI-powered applications quickly, reducing the time-to-market and improving the overall efficiency of the development process.\nBenefits of AI in Software Development\n Let us discuss what is the impact of AI on Software Development?\nImpact of AI in Software Development\nAI has significantly impacted software development by automating testing, debugging, and code optimization tasks. Well-organized and optimized code can improve the performance and accuracy of AI mod

{'topic': None,
 'messages': [ChatMessage(content='AI visionary, Grumpy old senior developer, Junior Software developer, discuss how the integration of AI tools in development processes specifically affects job roles and skill requirements for software developers in the current job market.', role='host'),
  ChatMessage(content='AI visionary, Grumpy old senior developer, Junior Software developer, discuss how the integration of AI tools in development processes specifically affects job roles and skill requirements for software developers in the current job market.', role='host'),
  ChatMessage(content='AI and automation have revolutionized software development. They streamline workflows, automate routine tasks, and improve the accuracy and efficiency of the development process. This leads to well-organized, optimized code, reduced development time, and cost, thus enhancing the overall quality of software.', role='You are AI visionary'),
  ChatMessage(content="AI and automation are tools